<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import psycopg2
import warnings

In [2]:
warnings.filterwarnings("ignore")

In [3]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [59]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [9]:
query_3_1 = f'''select count(*)
                from public.vacancies
'''

In [14]:
df = pd.read_sql_query(query_3_1, connection)
df

,count
0,49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [15]:
query_3_2 = f'''select count(*)
                from public.employers
'''

In [16]:
df = pd.read_sql_query(query_3_2, connection)
df

,count
0,23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [22]:
query_3_3 = f'''select count(*)
                from public.areas
'''

In [23]:
df = pd.read_sql_query(query_3_3, connection)
df

,count
0,1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [26]:
query_3_4 = f'''select count(*)
                from public.industries
'''

In [27]:
df = pd.read_sql_query(query_3_4, connection)
df

,count
0,294


***

Как видим, количество вакансий примерно в 2 раза превышает количество работодателей, что вполне ожидаемо, ведь, как правило, одной и той же организации для успешного функционирования требуются специалисты из разных областей. В базе представлено 1362 региона - это говорит о том, что мы охватываем довольно широкую географию. Также у нас имеются 294 различные сферы деятельности (т.е. в среднем по 167 вакансий на каждую сферу, но нужно, конечно, проводить дополнительный анализ). Это тоже довольно неплохая статистика, у соискателей есть возможность выбора и больше шансов куда-то устроиться в принципе (хотя, опять же, зависит от количества соискателей).

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [42]:
query_4_1 = f'''select 
                    a.name as area,
                    count(v.id) as vacancies_count
                from 
                    public.areas as a
                    join public.vacancies as v on v.area_id = a.id
                group by a.name
                order by count(v.id) desc
                limit 10
'''

In [43]:
df = pd.read_sql_query(query_4_1, connection)
df

,area,vacancies_count
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
5,Екатеринбург,1698
6,Нижний Новгород,1670
7,Казань,1415
8,Краснодар,1301
9,Самара,1144


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [44]:
query_4_2 = f'''select 
                    count(*)
                from 
                    public.vacancies
                where salary_from is not null
                    or salary_to is not null
'''

In [47]:
df = pd.read_sql_query(query_4_2, connection)
df

,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [50]:
query_4_3 = f'''select 
                    avg(salary_from) as mean_salary_from,
                    avg(salary_to) as mean_salary_to
                from 
                    public.vacancies
'''

In [53]:
df = pd.read_sql_query(query_4_3, connection)
print(round(df['mean_salary_from']))
print(round(df['mean_salary_to']))

0    71065.0
Name: mean_salary_from, dtype: float64
0    110537.0
Name: mean_salary_to, dtype: float64


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [64]:
query_4_4 = f'''select 
                    schedule,
                    employment,
                    count(*) as vacancies_count
                from 
                    public.vacancies
                group by schedule, employment
                order by count(*) desc
'''

In [65]:
df = pd.read_sql_query(query_4_4, connection)
df

,schedule,employment,vacancies_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [66]:
query_4_5 = f'''select 
                    experience,
                    count(*) as vacancies_count
                from 
                    public.vacancies
                group by experience
                order by count(*) 
'''

In [69]:
df = pd.read_sql_query(query_4_5, connection)
df

,experience,vacancies_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

Видим, что основная доля вакансий сосредоточена в крупных городах и столицах стран СНГ: в топ-5 по числу вакансий входят Москва, Санкт-Петербург, Минск, Новосибирск и Алматы (на данные города приходится примерно 30% всех вакансий). И это неудивительно, ведь именно в крупных городах расположено большинство предпрниятий и бизнес-центров. Примерно 72% вакансий имеют тип рабочего графика "полный день" и тип трудоустройства "полная занятость". Это тоже вполне ожидаемо, ведь работодателями выгоднее нанимать сотрудников, которые полностью будут погружены в дела компании и всегда будут присутствовать на своем рабочем месте. Наиболее частые пожелания работодателей по опыту работы (53% вакансий) - от 1 до 3 лет. Это вполне достаточно, чтобы погрузиться в предметную область и как следует в ней разобраться. Для ряда должностей, связанных с занимаением каких-либо руководящих позиций, может потребоваться уже больше опыта - от 3 до 6 лет (около 29% вакансий). Требование опыта более 6 лет кажется уже избыточным. Также можно заметить, что нижняя граница зарплатной вилки имеет довольно высокое среднее значение - более 70 тыс. рублей в месяц. Вероятно, это также связано с тем, что основная доля вакансий сосредоточена в крупных городах и столицах (например, по данным на начало 2022 года средняя зарплата в Москве составляла 116 354 рублей в месяц).

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [12]:
query_5_1 = f'''select 
                    e.name, 
                    count(v.id) as vacancies_count
                from 
                    public.employers as e
                    join public.vacancies as v on v.employer_id = e.id
                group by e.name
                order by count(v.id) desc
                limit 5
'''

In [13]:
df = pd.read_sql_query(query_5_1, connection)
df

,name,vacancies_count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [62]:
query_5_2 = f'''select 
                    a.name,
                    count(e.name) as employers_count
                from 
                    public.areas as a
                    left join public.vacancies as v on v.area_id = a.id
                    join public.employers as e on e.area = a.id
                group by a.name
                having count(v.id) = 0
                order by count(e.name) desc
                limit 10
                
'''

In [63]:
df = pd.read_sql_query(query_5_2, connection)
df

,name,employers_count
0,Россия,410
1,Казахстан,207
2,Московская область,75
3,Краснодарский край,19
4,Беларусь,18
5,Ростовская область,18
6,Азербайджан,17
7,Республика Татарстан,16
8,Нижегородская область,16
9,Узбекистан,15


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [ ]:
# текст запроса

In [ ]:
# результат запроса

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [ ]:
# код для получения списка городов-милионников

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [ ]:
# текст запроса

In [ ]:
# результат запроса

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [ ]:
# текст запроса

In [ ]:
# результат запроса

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [ ]:
# текст запроса

In [ ]:
# результат запроса

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [ ]:
# текст запроса

In [ ]:
# результат запроса

***

In [ ]:
# выводы по предметному анализу

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования